Overview:
Data Capture
Data learn

In [1]:
import numpy as np
import cv2 as cv2
from mss import mss
from PIL import Image, ImageEnhance, ImageOps
import keyboard
import time
import tqdm as tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf                                                               
import random
from tqdm import tqdm
from tensorflow.keras.models import model_from_json

In [2]:
class Agent:
    def __init__(self):
        #This is the actual Neural net
        model = Sequential([ 
            Conv2D(32, (8,8), input_shape=(76, 384, 4),
                   strides=(2,2), activation='relu'),
            MaxPooling2D(pool_size=(5,5), strides=(2, 2)),
            Conv2D(64, (4,4), activation='relu', strides=(1,1)),
            MaxPooling2D(pool_size=(7, 7), strides=(3, 3)),
            Conv2D(128, (1, 1), strides=(1,1), activation='relu'),
            MaxPooling2D(pool_size=(3,3), strides=(3,3)),
            Flatten(),
            Dense(384, activation='relu'),
            Dense(64, activation="relu", name="layer1"),
            Dense(8, activation="relu", name="layer2"),
            Dense(3, activation="linear", name="layer3"),
        ])
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001)) 
        # model.load_weights("DinoGameSpeed4.h5")
        self.model = model
        self.memory = []
        # print(self.model.summary()) 
        self.xTrain = []
        self.yTrain = []
        self.loss = []
        self.imageBank = []
        self.imageBankLength = 4 #number of frames for the conv net
        #image processing
        self.ones = np.ones((76,384,4))
        self.zeros = np.zeros((76,384,4))  
        self.zeros1 = np.zeros((76,384,4))
        self.zeros2 = np.zeros((76,384,4))
        self.zeros3 = np.zeros((76,384,4))
        self.zeros4 = np.zeros((76,384,4))
        self.zeros1[:,:,0] = 1
        self.zeros2[:,:,1] = 1
        self.zeros3[:,:,2] = 1
        self.zeros4[:,:,3] = 1


    def act(self, state):
#TODO: add image bank stuff
        action = self.predict(state)
        return action

    # This function stores experiences in the experience replay
    def remember(self, state, action):
        state = self._processImg(state)
        self.location = location
        self.memory.append(np.array([state, action]))

    #TODO: finish funciton
    def generateData(self, batch):
        return None
        #image flip
        #change brightness

    def learn(self):
        batch = np.array(self.memory)
        batch = self.generateData(batch)
        states = batch[:, 0].tolist()
        actions = batch[:, 1].tolist()
        
        history = self.model.fit(
            states, actions, batch_size=32, epochs=1, verbose=0)
        loss = history.history.get("loss")[0]
        print("LOSS: ", loss)
        self.loss.append(loss)
        self.xTrain = []
        self.yTrain = []

            
    def _processImg(self, img):
        img = Image.fromarray(img)
        img = img.resize((384, 76), Image.ANTIALIAS)
        img = self._contrast(img)

        #You can use the following open CV code segment to test your in game screenshots
        # cv2.imshow("image",img)
        # if cv2.waitKey(25) & 0xFF == ord('q'): 
        #     cv2.destroyAllWindows()
        
        img = np.reshape(img, (76,384))
        return img

    def _contrast(self,pixvals):
        minval = 32 #np.percentile(pixvals, 2)
        maxval = 171 #np.percentile(pixvals, 98)
        pixvals = np.clip(pixvals, minval, maxval)
        pixvals = ((pixvals - minval) / (maxval - minval))
        return pixvals

    def _imageBankHandler(self, img):
        img = np.array(img)
        while len(self.imageBank) < (self.imageBankLength): 
            self.imageBank.append(np.reshape(img,(76,384,1)) * self.ones)

        bank = np.array(self.imageBank)
        toReturn = self.zeros
        img1 = (np.reshape(img,(76,384,1)) * self.ones)  * self.zeros1
        img2 = bank[0] * self.zeros2
        img3 = bank[1] * self.zeros3
        img4 = bank[2] * self.zeros4


        toReturn = np.array(img1 + img2 + img3 + img4)        

        self.imageBank.pop(0)
        self.imageBank.append(np.reshape(img,(76 ,384,1)) * self.ones)

        return toReturn

In [4]:
agent = Agent() #currently agent is configured with only 2 actions
while True: #CHANGE to while button pressed or something
    #TODO: Get state
    action = agent.observe(state) #TODO change to pwm read
    agent.remember(state, action)
agent.learn()
agent.model.save_weights ("selfdrive.h5")

while True: #CHANGE to while button pressed or something
    #TODO: Get state
    action = agent.act(state) #TODO change to pwm read


game starting in  2
game starting in  1
game starting in  0
  0%|          | 0/3500 [00:00<?, ?it/s]breaking
Avg Frame-Rate:  17.377813788984955
900
  0%|          | 1/3500 [00:54<53:05:14, 54.62s/it]LOSS:  1.433573842048645
Saved model to disk
  0%|          | 1/3500 [01:23<81:29:35, 83.85s/it]


KeyboardInterrupt: 

This next section plots the AI's total reward and the Nueral net learning loss for the AI as a function of the episode number